# Finding prices using LLMs and Langgraph

The objective on this notebook is to find a price from some object from specific URL's, in the notebook example, we will use the [zoom](https://www.zoom.com.br).

For this task, we gonna use Langgraph and Langchain, to divide this task in some particular tasks. 

---

Importing packages:

In [ ]:
from dotenv import load_dotenv

from langchain_openai.chat_models import ChatOpenAI

from langchain_community.document_loaders import WebBaseLoader

from langchain_core.messages import (
    AnyMessage, 
    AIMessage, 
    HumanMessage, 
    SystemMessage
)

from langgraph.graph import StateGraph, START, END, MessagesState

from re import search

from os import getenv

from typing import Literal

Configurations:

In [ ]:
load_dotenv()

In [ ]:
OPEN_AI_URL = getenv("OPENAI_URL")
OPEN_AI_MODEL = getenv("MODEL")

In [ ]:
RETAIL_URL = "https://www.zoom.com.br"

---

First, we need to define a way to use the retail store url to search some prices!

In [ ]:
RETAIL_SUFIX = lambda input: f"{RETAIL_URL}/search?q={input.replace(' ','%20')}"

Now, let's consume the first retail for some respective input. For this, we will use 

In [ ]:
INPUT = "ar condicionado"

In [ ]:
url_search = RETAIL_SUFIX(INPUT)

In [ ]:
url_search

In [ ]:
web_loaded = WebBaseLoader(url_search)

In [ ]:
[document] = web_loaded.load()

Let's dive in on what is on the document!

In [ ]:
document.dict().keys()

In [ ]:
document.id

In [ ]:
document.metadata

In [ ]:
print(document.page_content)

In [ ]:
document.type

Now, we can use a textual content in a LLM!

But before, let's find a way to delimiter the retails on the search!

In [ ]:
content_split_lines = document.page_content.splitlines()

In [ ]:
RETAILS = [
    "Casas Bahia",
    "Extra",
    "Magazine Luiza"   
]

In [ ]:
content_split_lines

In [ ]:
pattern = rf'({"|".join(RETAILS)})'

pattern

In [ ]:
lines_with_retails = [content.strip() for content in content_split_lines if search(pattern, content)]

In [ ]:
lines_with_retails

Now, let's send this to a LLM!

In [ ]:
chat = ChatOpenAI(base_url=OPEN_AI_URL, model=OPEN_AI_MODEL)

In [ ]:
breakline = lambda: '\n'

messages: list[AnyMessage] = [
    SystemMessage(content="Você é um assistente especializado em ler uma série de ofertas com diferentes preços e trazer a melhor oferta e a sua respectiva loja!", name="System"),
    HumanMessage(content=f"Me dê a melhor oferta a partir das seguintes opções:\n{breakline().join(lines_with_retails)}\nNão esqueça de me falar onde comprar", name="Author")
]

In [ ]:
answer = chat.invoke(messages)

In [ ]:
print(answer.content)

All right! Now we understand how to use the ``WebBaseLoader`` to load websites and take text from it!